In [9]:
#Importing Important Packages

In [10]:
import face_recognition as fr
from pathlib import Path as path
import cv2 as cv
from numba import jit, cuda

In [11]:
#Checking the paths:

In [18]:
# p = path('.')
# [x for x in p.iterdir() if x.is_dir()]
Known_Faces = path('Known_Faces')
Unknown_Faces = path('Unknown_Faces')
# Known_Me_path = Known_Faces /'Mahmoud'
# Known_Lud_path = Known_Faces /'Ludwig'

In [19]:
#Setting the Tolearnace: it basically allows us to either risk getting a match everytime with a high chance of it being false
# or not to get a match but everytime it matches its more accurate
TOLERACNCE = 0.6

In [20]:
#Box Settings:
FRAME_THICKNESS= 2
FONT_THICKNESS=1

In [21]:
#Model Settings: you could use hog too.
MODEL = 'cnn' #is a class of neural networks that 
#specializes in processing data that has a grid-like topology, such as an image.

In [22]:
#Load Known Faces:
print("Loading Known Faces")
known_faces = []
known_names = []
for name in Known_Faces.iterdir(): 
    #Remove the .ipynb_checkpoints.
    if not str(name).startswith('Known_Faces\.'): 
#         print(name)
        for filename in name.iterdir():
            #print(filename)
            #Load Image
            image = fr.load_image_file(filename)
            #Index 0, so it only takes the first face it finds in the image as the person we are interested in.    
            encoding = fr.face_encodings(image)[0]
            known_faces.append(encoding)
#             print('before: ', name)
            stripped_name = str(name).strip("\Known_Faces")
            known_names.append(stripped_name)
#             print('after: ', stripped_name)

Loading Known Faces


In [23]:
#Load Unknown Faces:
print("Checking Unknown Faces")
for filename in Unknown_Faces.iterdir():
#     print(filename)
    image= fr.load_image_file(filename)
    locations = fr.face_locations(image, model=MODEL)
    encodings = fr.face_encodings(image,locations)
    image = cv.cvtColor(image,cv.COLOR_RGB2BGR)
        #Compare the known faces with the encoding of the unknown and then we will get a lits of booleans.
    #Show the Image and the name
    for face_encoding , face_location in zip(encodings,locations):
        results = fr.compare_faces(known_faces,face_encoding,TOLERACNCE)
        match = None
        if True in results:
            match = known_names[results.index(True)]
            print('Match found:', match)
            #To draw a rect u just need the top left and bottom right cordinates.
            top_left = (face_location[3],face_location[0])
            bottom_right = (face_location[1],face_location[2])
            color = [255,255,255]
            cv.rectangle(image,top_left,bottom_right,color, FRAME_THICKNESS)
            cv.putText(image,match,(face_location[3],face_location[2]+10), 
                        cv.FONT_HERSHEY_DUPLEX, 0.5, (0,255,0), FONT_THICKNESS)
        else:
            print('no Match Found')
            top_left = (face_location[3],face_location[0])
            bottom_right = (face_location[1],face_location[2])
            color = [0,0,255]
            cv.rectangle(image,top_left,bottom_right,color, FRAME_THICKNESS)
            cv.putText(image,'no match',(face_location[3],face_location[2]+10), 
                        cv.FONT_HERSHEY_DUPLEX, 0.5, (0,255,0), FONT_THICKNESS)
                
    cv.imshow("",image)
    cv.waitKey(0)
    cv.destroyAllWindows()

Checking Unknown Faces
no Match Found
Match found: Ludwig
no Match Found
Match found: Ludwig
Match found: Ludwig
no Match Found
Match found: Ludwig
Match found: Mahmoud
Match found: Mahmoud
Match found: Mahmoud
no Match Found
no Match Found
no Match Found
no Match Found
no Match Found
no Match Found
no Match Found


In [ ]:
#Video FR :
#Comment out Unknow_faces, because the video is going to be the unknown faces.

In [ ]:
cam = cv.VideoCapture(1)
# cam.set(3,640)
# cam.set(4,480)
# cv.namedWindow("test")

#Similar to the unknown faces part, but instead of the images folder, use the video feed:

#Load Unknown Faces:
# print("Checking Unknown Faces")
while True:
#   print(filename)
    ret, frame = cam.read()
    locations = fr.face_locations(frame, model=MODEL)
    encodings = fr.face_encodings(frame,locations)
    #Compare the known faces with the encoding of the unknown and then we will get a lits of booleans.
    #Show the Image and the name
    for face_encoding , face_location in zip(encodings,locations):
        results = fr.compare_faces(known_faces,face_encoding,TOLERACNCE)
        match = None
        if True in results:
            match = known_names[results.index(True)]
#             print('Match found:', match)
            #To draw a rect u just need the top left and bottom right cordinates.
            top_left = (face_location[3],face_location[0])
            bottom_right = (face_location[1],face_location[2])
            color = [255,255,255]
            cv.rectangle(frame,top_left,bottom_right,color, FRAME_THICKNESS)
            cv.putText(frame,match,(face_location[3],face_location[2]+10), 
                        cv.FONT_HERSHEY_DUPLEX, 0.5, (0,255,0), FONT_THICKNESS)
        else:
#             print('no Match Found')
            top_left = (face_location[3],face_location[0])
            bottom_right = (face_location[1],face_location[2])
            color = [0,0,255]
            cv.rectangle(frame,top_left,bottom_right,color, FRAME_THICKNESS)
            cv.putText(frame,'no match',(face_location[3],face_location[2]+10), 
                        cv.FONT_HERSHEY_DUPLEX, 0.5, (0,255,0), FONT_THICKNESS)
            
    cv.imshow("",frame)
    k = cv.waitKey(1)
    if k%256 == 27:
        #esc
        print("Closing")
        break
cv.waitKey(0)
cv.destroyAllWindows()